In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
imgPaths = glob('./Selected/*.jpeg')

In [8]:
imgPaths[0]
print(os.path.split(imgPaths[0]))

('./Selected', '000.jpeg')


In [10]:
allBusinessCard = pd.DataFrame(columns=['id','text'])

for imgPath in  tqdm(imgPaths,desc='BusinessCard'):
    
    #imgPath = imgPaths[0]
    _, filename = os.path.split(imgPath)
    # extract data and text 
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image)
    dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
    df = pd.DataFrame(dataList[1:],columns=dataList[0])
    df.dropna(inplace=True)
    df['conf'] = df['conf'].astype(float).astype(int)

    useFulData = df.query('conf >= 30')

    # Dataframe
    businessCard = pd.DataFrame()
    businessCard['text'] = useFulData['text']
    businessCard['id'] = filename
    
    # concatenation
    allBusinessCard = pd.concat((allBusinessCard,businessCard))

BusinessCard: 100%|██████████████████████████████████████████████████████████████████| 293/293 [06:12<00:00,  1.27s/it]


In [11]:
allBusinessCard.to_csv('businessCard.csv',index=False)